<a href="https://colab.research.google.com/github/akvo/usaid-wssh-tool-3/blob/main/scripts/data-processing-collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
# @title Prologue
%%capture
# 1. Import Dependencies
import geopandas as gpd
import pandas as pd
import requests
from io import StringIO
import os
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from IPython.display import display, clear_output, IFrame
from folium import Choropleth
from IPython.display import IFrame
import requests as req

# 2. Clone Repository
if not os.path.exists("usaid-wssh-tool-3"):
  !git clone https://github.com/akvo/usaid-wssh-tool-3.git

geojson_path = "usaid-wssh-tool-3/data/output"
file_list = [f for f in os.listdir(geojson_path) if os.path.isfile(os.path.join(geojson_path, f))]
file_list = list(filter(lambda x: x.endswith(".geojson"), file_list))

# 3. Column Definitions
column_definitions = {
    'drr': 'Weighted Drought Risk',
    'rfr': 'Weighted Riverine Flood Risk',
    'bws' : 'Weighted Base Water Stress',
    'Open_defecation_estimates_mean': 'Open Defecation',
    'No_Improved_water_premise_estimates_mean': 'No Improve Water Premises',
    'No_basic_water_estimates_mean': 'No Basic Water'
}
column_list = list(column_definitions.keys())
weight_mapping = {'core': 0.7, 'secondary': 0.3}

In [77]:
# @title Init Map Functions

# Apply Directionality
def apply_directionality(data, indicators, directionality):
    directed_data = {}
    for idx, indicator in enumerate(indicators):
        if directionality[indicator] == 'positive':
            directed_data[indicator] = data[indicator]  # Positive direction, keep the value as is
        else:
            directed_data[indicator] = 1 - data[indicator]  # Negative direction, invert the value (1 - value)
    return directed_data

# Get center of the maps
def get_center_cordinates(geojson):
    center_x = (geojson.bounds['minx'].min() + geojson.bounds['maxx'].max()) / 2
    center_y = (geojson.bounds['miny'].min() + geojson.bounds['maxy'].max()) / 2
    return [center_y, center_x]

# Function to calculate the weighted index and show on the map
def calculate_index(countries, indicators, core_secondary, directionality):
    merged_geojson = []
    for country in countries:
        # use saved transformed data if available
        if os.path.exists(f"{geojson_path}/{country.lower()}_transformed.geojson"):
            geojson = gpd.read_file(f"{geojson_path}/{country.lower()}_transformed.geojson")
        else:
            geojson = gpd.read_file(f"{geojson_path}/{country.lower()}.geojson")

            # Initialize the MinMaxScaler
            missing_columns = [col for col in column_list if col not in geojson.columns]
            for col in missing_columns:
              print(f"Adding missing column: {col}")
              geojson[col] = 0

            scaler = MinMaxScaler()

            # Apply Min-Max scaling to the WASH indicator columns
            geojson[column_list] = scaler.fit_transform(geojson[column_list])

            # save transformed data to a temporary json
            geojson.to_file(f"{geojson_path}/{country.lower()}_transformed.geojson", driver='GeoJSON')

        # Apply core/secondary
        geojson['index'] = 0
        directed_data = apply_directionality(geojson, indicators, directionality)
        # Apply core/secondary and directionality
        merged_geojson.append(geojson)

    geojson = gpd.GeoDataFrame(pd.concat(merged_geojson, ignore_index=True))


    for indicator in indicators:
        weight = weight_mapping[core_secondary[indicator]]
        geojson['index'] += directed_data[indicator] * weight

    # Clear the map area to prevent stacking maps
    with map_area:
        clear_output(wait=True)

    # Create a choropleth map with folium
    center_cordinate = get_center_cordinates(geojson)
    m = folium.Map(location=center_cordinate, zoom_start=7)

    Choropleth(
        geo_data=geojson.to_json(),
        data=geojson,
        columns=["ADM2_EN", "index"],
        key_on="feature.properties.ADM2_EN",
        fill_color="YlGnBu",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="WASH Index"
    ).add_to(m)

    # Add a tooltip to display the index score on hover
    folium.GeoJson(
        geojson,
        style_function=lambda x: {'fillColor': '#ffffff00', 'color': '#00000000', 'weight': 0},
        tooltip=folium.GeoJsonTooltip(fields=["ADM2_EN", "index"],
                                      aliases=["Region: ", "Index Score: "],
                                      localize=True)
    ).add_to(m)

    # Add a custom legend with "Low priority" and "High priority" labels
    legend_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 150px; height: 90px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     padding: 10px;">
     <b>Priority Legend</b><br>
     <i style="background: #ffffcc; width: 20px; height: 10px; display: inline-block;"></i> Low priority<br>
     <i style="background: #08306b; width: 20px; height: 10px; display: inline-block;"></i> High priority<br>
     </div>
     '''
    m.get_root().html.add_child(folium.Element(legend_html))

    display(m)

In [100]:
# @title Select Countries & Indicators
# 1. Init Container
output_area = widgets.Output()  # For buttons and widgets
map_area = widgets.Output()     # For the map

core_secondary_title_widgets = {}
core_secondary_widgets = {}
directionality_widgets = {}


output_container = []

# 2. Country selection widget
country_list = list(map(lambda x: x.replace(".geojson", ""), file_list))
country_list.sort()
country_list = [c.capitalize() for c in country_list]
country_selection = widgets.SelectMultiple(
    options=[c.capitalize() for c in country_list],
    description='Countries',
    value=[country_list[0]],
    disabled=False,
    layout=widgets.Layout(width='500px', height='200px')  # Set custom width and height
)

# 3. Indicator Selection
indicator_selection = widgets.SelectMultiple(
    options=column_list,
    description='Indicators',
    disabled=False,
    layout=widgets.Layout(width='500px', height='200px')  # Set custom width and height
)

# 4. Update Maps Button
update_button = widgets.Button(description="Calculate Index")

# 5. Function to handle the button click event for calculating the index
def on_calculate_button_click(b):
    selected_indicators = list(indicator_selection.value)
    selected_countries = list(country_selection.value)

    # Collect core/secondary and directionality choices
    core_sec = {indicator: core_secondary_widgets[indicator].value for indicator in selected_indicators}
    directionality = {indicator: directionality_widgets[indicator].value for indicator in selected_indicators}

    calculate_index(selected_countries, selected_indicators, core_sec, directionality)

update_button.on_click(on_calculate_button_click)

# 6. Function on change country
def on_change_country(change):
    clear_output(wait=True)
    output_container.clear()
    core_secondary_widgets.clear()
    directionality_widgets.clear()
    selected_countries = list(country_selection.value)

# 7. Function on change indicator
def on_change_indicator(change):

    clear_output(wait=True)
    output_container.clear()
    core_secondary_widgets.clear()
    directionality_widgets.clear()
    selected_indicators = list(indicator_selection.value)
    selected_countries = list(country_selection.value)
    # line widget
    output_container.append(widgets.HTML(value="<hr>"))
    for indicator in selected_indicators:
        indicator_name = column_definitions[indicator]
        if indicator not in core_secondary_widgets:
            core_secondary_widgets[indicator] = widgets.Dropdown(
                options=['core', 'secondary'],
                description=f'{indicator_name} :',
                value='core',
                layout=widgets.Layout(width='500px'),
                style={'description_width': 'initial'}
            )

        if indicator not in directionality_widgets:
            directionality_widgets[indicator] = widgets.Dropdown(
                options=['positive', 'negative'],
                description=f'{indicator_name} - Direction:',
                value='positive',
                layout=widgets.Layout(width='500px'),
                style={'description_width': 'initial'}
            )

        # Add both widgets (Core/Secondary and Directionality) to a VBox
        output_container.append(widgets.VBox([core_secondary_widgets[indicator], directionality_widgets[indicator]]))
    return display(country_selection, indicator_selection, widgets.VBox(output_container), update_button)

# 8. Attach an observer to capture changes in selection
country_selection.observe(on_change_country, names="value")
indicator_selection.observe(on_change_indicator, names="value")

display(country_selection, indicator_selection)

SelectMultiple(description='Countries', index=(0, 3), layout=Layout(height='200px', width='500px'), options=('…

SelectMultiple(description='Indicators', index=(3,), layout=Layout(height='200px', width='500px'), options=('d…

Button(description='Calculate Index', style=ButtonStyle())